<a href="https://colab.research.google.com/github/Sharon-Tseng/VincentMobile_Database_System/blob/c%26m/ProjectDemo_c%26m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer and Member

In [ ]:
# -- Copy and paste the whole code when doing project--
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
# create a link
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle

In [ ]:
# -- Copy and paste the whole code when doing project, using personal account for testing --
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "LA302"
PASSWORD = "CJ995"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)

c = conn.cursor()   # c: represent database connection

In [ ]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from datetime import date, datetime
import re

In [ ]:
from logging import disable
from pickle import FALSE
# tab object
tb_all = TabBar(['Introduction','Register',
          'Login','Welcome','Forget_Password',
          'Change_Password','Update_Personal_Account',
          'Search_Product','Show_Product_Detail'])

def showPage(tabName):
  with tb_all.output_to(tabName):
    if tabName == "Search_Product":
      show_records()
    display()

# Input validations:
def is_valid_username(username):
  return re.match(r"^[a-zA-Z0-9]{3,30}$",username)

def is_valid_name(name): # firstname_textbox.value, lastname_textbox.value
  return re.match(r"^[a-zA-Z0-9]{1,50}$",name)

def is_valid_pwd(pwd):
  return 6 <= len(pwd) <= 30

def is_valid_address(address):
  return 1 <= len(address) <= 100

def is_valid_answer(security_answer):
  return re.match(r"^[a-zA-Z0-9]{1,50}$",security_answer)

def is_valid_creditno(credit_no):
  return re.match(r"^[0-9]{8,20}$",credit_no)

def is_valid_cvc(cvc):
  return re.match(r"^[0-9]{3,4}$",cvc)

def is_valid_phone(phone):
  return re.match(r"^[0-9]{8,8}$",phone)

def is_valid_email(email):
  return re.match(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$", email)

##############################################
############## Introduction Tab ###########
##############################################

with tb_all.output_to('Introduction'):
  intro_label = Label("Welcome to 302 Vincent Mobile")

  btn_RegisterPage = widgets.Button(description="Register as Member")
  btn_LoginPage = widgets.Button(description="Login")

  def on_btnRegisterPage_button_clicked(b):
    showPage("Register")
  btn_RegisterPage.on_click(on_btnRegisterPage_button_clicked)

  def on_btn_LoginPage_button_clicked(b):
    showPage("Login")
  btn_LoginPage.on_click(on_btn_LoginPage_button_clicked)

  intro_row1HBox = widgets.HBox([intro_label])
  intro_row2HBox = widgets.HBox([btn_LoginPage])
  intro_row3HBox = widgets.HBox([btn_RegisterPage])

  display(widgets.VBox([intro_row1HBox, intro_row2HBox, intro_row3HBox]))

##############################################
############## Register Tab ###############
##############################################

with tb_all.output_to("Register"):
  reg_label = Label("Please fill in the information")
  username_label = Label("Username: ")
  username_textbox = Text(placeholder='Enter your username', disabled=False)
  password_label = Label("Password: ")
  password_textbox = Text(placeholder='Enter your password', disabled=False)
  firstname_label = Label("First Name: ")
  firstname_textbox = Text(placeholder='Tai Ming', disabled=False)
  lastname_label = Label("Last Name: ")
  lastname_textbox = Text(placeholder='Wong', disabled=False)
  address_label = Label("Address: ")
  address_textbox = Text(placeholder='Sai Kung, Hong Kong', disabled=False)
  gender_label = Label("Gender: ")
  gender_dropdown = widgets.Dropdown(options=["Male", "Female", "Others"], value="Male")
  dob_label = Label("Date of Birth: ")
  dob_datepicker = DatePicker(placeholder='2000/01/01', disabled=False, value= date.today())
  phone_label = Label("Phone: ")
  phone_textbox = Text(placeholder='12345678', disabled=False)
  email_label = Label("Email: ")
  email_textbox = Text(placeholder='e.g, example@domain.com', disabled=False)
  creditno_label = Label("Credit No.: ")
  creditno_textbox = Text(placeholder='1234567891234567', disabled=False)
  security_question_label = Label("Security Question: ")
  cmb = widgets.Dropdown(options=["What is your favorite animal?", "Which city did you live when you were 10?",
                    "What year is your grandmother born?", "What is the name of your first pet?",
                    "What is your favorite sport?"], value = "What is your favorite animal?")
  security_answer_label = Label("Security Answer: ")
  security_answer_textbox = Text(placeholder='Security Answer', disabled=False)
  expiry_date_label = Label("Expiry Date: ")
  expiry_date_datepicker = DatePicker(dplaceholder='1991-01-01', disabled=False, value= date.today())
  cvc_label = Label("CVC: ")
  cvc_textbox = Text(placeholder='000', disabled=False)

  reg_status = Label("")
  reg_register_button = widgets.Button(description="Register")
  reg_back_button = widgets.Button(description="Back")


  # Register Button - event handler
  def on_register_button_clicked(b):

    all_input_values = [username_textbox.value, password_textbox.value, firstname_textbox.value, lastname_textbox.value, address_textbox.value,
             phone_textbox.value, email_textbox.value, creditno_textbox.value, cvc_textbox.value, security_answer_textbox.value]

    # check null values
    for i in all_input_values:
      if i == "":
        reg_status.value = "❗Missing Value. Please input all items"
        return

    try:
      if not is_valid_username(username_textbox.value):
        reg_status.value = "❗Username must be 3-30 characters, letters and numbers only."
        return

      if not is_valid_pwd(password_textbox.value):
        reg_status.value = "❗Password must be 6-30 characters, letters and numbers only."
        return

      if not is_valid_name(firstname_textbox.value):
        reg_status.value = "❗Firstname must be 1-50 characters only."
        return

      if not is_valid_name(lastname_textbox.value):
        reg_status.value = "❗Lastname must be 1-50 characters only."
        return


      if not is_valid_address(address_textbox.value):
        reg_status.value = "❗Address must be within 100 characters."
        return

      if not is_valid_phone(phone_textbox.value):
        reg_status.value = "❗Phone must be 8 numbers only."
        return

      if not is_valid_email(email_textbox.value):
        reg_status.value = "❗Invalid email format. Please use example@domain.com"
        return

      if not is_valid_creditno(creditno_textbox.value):
        reg_status.value = "❗Credit card number must be 8-20 numbers only."
        return

      if not is_valid_cvc(cvc_textbox.value):
        reg_status.value = "❗CVC must be 3-4 numbers only."
        return

      if not is_valid_answer(security_answer_textbox.value):
        reg_status.value = "❗Security answer must be 1-50 characters and numbers only."
        return


      # exclude duplicated username
      c.execute("SELECT M_USERNAME FROM MEMBER WHERE M_USERNAME = '{}'".format(username_textbox.value))
      existing_user = c.fetchone()

      if existing_user:
        reg_status.value = "Username already exists. Please create another username."
        return

        username_textbox.value = ""
        password_textbox.value = ""
        firstname_textbox.value = ""
        lastname_textbox.value = ""
        address_textbox.value = ""
        phone_textbox.value = ""
        email_textbox.value = ""
        creditno_textbox.value = ""
        security_answer_textbox.value=""
        cvc_textbox.value=""


      else:
        c.execute(
        """
        INSERT INTO MEMBER VALUES(seq_member.nextval, '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}' ,TO_DATE('{}', 'YYYY-MM-DD'), '{}', '{}', '{}', TO_DATE('{}', 'YYYY-MM-DD'), '{}')
        """.format(firstname_textbox.value, lastname_textbox.value,
              username_textbox.value, password_textbox.value, address_textbox.value,
              cmb.value, security_answer_textbox.value, creditno_textbox.value,
              expiry_date_datepicker.value, cvc_textbox.value, phone_textbox.value, email_textbox.value,
              dob_datepicker.value, gender_dropdown.value)
        )

        c.execute("commit")

        username_textbox.value = ""
        password_textbox.value = ""
        firstname_textbox.value = ""
        lastname_textbox.value = ""
        address_textbox.value = ""
        phone_textbox.value = ""
        email_textbox.value = ""
        creditno_textbox.value = ""
        security_answer_textbox.value=""
        cvc_textbox.value=""


        reg_register_button.disabled=True
        reg_status.value = "Successfully Registered!"

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      reg_status.value = f"Database Error:{error_obj.message}"

  reg_register_button.on_click(on_register_button_clicked)

  def on_back_button_clicked(b):
    showPage("Introduction")
  reg_back_button.on_click(on_back_button_clicked)

  # display objects
  reg_msg_labelVBox = widgets.VBox([reg_label])
  reg_labelVBox = widgets.VBox([username_label, password_label, firstname_label, lastname_label,
                  address_label,gender_label,dob_label,phone_label,email_label,
                  creditno_label,cvc_label,security_question_label,security_answer_label,expiry_date_label])
  reg_textboxVBox = widgets.VBox([username_textbox, password_textbox, firstname_textbox, lastname_textbox,
                  address_textbox,gender_dropdown,dob_datepicker,phone_textbox,email_textbox,
                  creditno_textbox,cvc_textbox,cmb,security_answer_textbox,expiry_date_datepicker])
  reg_line1HBox = widgets.HBox([reg_msg_labelVBox])
  reg_line2HBox = widgets.HBox([reg_labelVBox, reg_textboxVBox])
  reg_line3HBox = widgets.HBox([reg_register_button, reg_back_button])
  reg_line4HBox = widgets.HBox([reg_status])

  display(widgets.VBox([reg_line1HBox,reg_line2HBox, reg_line3HBox,reg_line4HBox]))

##############################################
############## Login Tab ##################
##############################################

with tb_all.output_to('Login'):
  login_label = Label("Please login with your username and password:")
  login_un_label = Label("Username: ")
  login_un_textbox = Text(placeholder='Enter your username', disabled=False)
  login_pwd_label = Label("Password: ")
  login_pwd_textbox = Text(placeholder='Enter your password', disabled=False)

  login_status = Label("")
  login_button = widgets.Button(description="Login")
  forget_pwd_button = widgets.Button(description="Forget Password")
  login_back_button = widgets.Button(description="Back")

  member_username = widgets.Label("")
  member_pwd = widgets.Label("")
  mid_label = widgets.Label("")

  def on_button_login_clicked(b):
    c.execute(
    """
    SELECT M_ID
    FROM MEMBER
    WHERE EXISTS (SELECT M_USERNAME, M_PW FROM MEMBER WHERE M_USERNAME = '{}' AND M_PW = '{}')
    """.format(login_un_textbox.value, login_pwd_textbox.value)
    )

    member_username.value = login_un_textbox.value
    member_pwd.value = login_pwd_textbox.value

    login_un_textbox.value = ""
    login_pwd_textbox.value = ""

    row = c.fetchone()

    if row:
      member_id = row[0]
      showPage("Welcome")
    else:
      login_status.value = "❗Incorrect username or password! Please try again."
      showPage("Login")

    mid_label.value = str(row[0])

  login_button.on_click(on_button_login_clicked)

  def on_back_button_clicked(b):
    showPage("Introduction")
  login_back_button.on_click(on_back_button_clicked)

  # design login interface
  login_msg_labelVBox = widgets.VBox([login_label])
  login_labelVBox = widgets.VBox([login_un_label,login_pwd_label])
  login_textboxVBox = widgets.VBox([login_un_textbox,login_pwd_textbox])

  login_line1HBox = widgets.HBox([login_msg_labelVBox])
  login_line2HBox = widgets.HBox([login_labelVBox, login_textboxVBox])
  login_line3HBox = widgets.HBox([login_button, forget_pwd_button, login_back_button])
  login_line4HBox = widgets.HBox([login_status])

  display(widgets.VBox([login_line1HBox, login_line2HBox, login_line3HBox, login_line4HBox]))

  def on_forget_pwd_clicked(b):
    showPage("Forget_Password")
  forget_pwd_button.on_click(on_forget_pwd_clicked)

##############################################
########## Forget_Password Tab ############
##############################################

with tb_all.output_to('Forget_Password'):
  fpwd_label = Label("Please fill in the below information to verify your identity:")
  fpwd_un_textbox = Text(value='', placeholder='Enter your username', description='Username:', disabled=False)
  spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  fpwd_question_label = Label("Security Question")
  fpwd_question_textbox = Text(placeholder='e.g, What is your favorite animal?', disabled = True)
  fpwd_answer_label = Label("Security Question Answer")
  fpwd_ans_textbox = Text(value='',placeholder='e.g, dog', disabled=False)

  fpwd_status = Label("")
  member_username_2 = widgets.Label("")

  retrieve_button = widgets.Button(description="Retrieve")
  back_button = widgets.Button(description="Back")
  confirm_button = widgets.Button(description="Confirm")

  # retrieve button
  def on_retrieve_button_clicked(b):
    if fpwd_un_textbox.value == "":
      fpwd_status.value = "❗Please enter a valid username!"
    else:
      c.execute("SELECT M_QUESTION FROM MEMBER WHERE M_USERNAME = '{}'".format(fpwd_un_textbox.value))

      row = c.fetchone()

      if row:
        if row[0] is not None:
          fpwd_question_textbox.value = row[0]
          on_retrieve_button_clicked.disabled = True
          member_username_2.value = fpwd_un_textbox.value
          fpwd_status.value = ""
      else:
        fpwd_status.value = "❗Username not found"
  retrieve_button.on_click(on_retrieve_button_clicked)

  def on_confirm_button_clicked(b):
    c.execute(
    """
    SELECT M_ANSWER FROM MEMBER
    WHERE M_USERNAME = '{}' AND M_QUESTION = '{}'
    """.format(fpwd_un_textbox.value,fpwd_question_textbox.value))

    for row in c:
      if fpwd_ans_textbox.value == row[0]:
        showPage('Change_Password')
      else:
        fpwd_status.value = "❗Incorrect answer! Please try again."

    fpwd_un_textbox.value = ""
    fpwd_question_textbox.value = ""
    fpwd_ans_textbox.value= ""

  confirm_button.on_click(on_confirm_button_clicked)


  # back button
  def on_back_button_clicked(b):
    showPage("Login")
  back_button.on_click(on_back_button_clicked)

  line1HBox = widgets.HBox([fpwd_label])
  line2HBox = widgets.HBox([fpwd_un_textbox, retrieve_button])
  line3HBox = widgets.HBox([fpwd_status])
  line4HBox = widgets.HBox([spacer])
  line5HBox = widgets.HBox([fpwd_question_label, fpwd_question_textbox])
  line6HBox = widgets.HBox([fpwd_answer_label, fpwd_ans_textbox])
  line7HBox = widgets.HBox([back_button, confirm_button])

  display(widgets.VBox([line1HBox,line2HBox,line4HBox,line5HBox,line6HBox,line7HBox,line3HBox]))

##############################################
########## Change PWD Tab ################
##############################################

with tb_all.output_to('Change_Password'):
  cpwd_label = Label("Please fill in the below information to reset your password:")
  new_pwd_label = Label("New Password:")
  new_pwd_textbox = Text(placeholder='Your new password', disabled=False)
  retype_label = Label("Retype:")
  retype_textbox = Text(placeholder='Retype your new password', disabled=False)

  cpwd_msg_label = Label("")
  update_button = widgets.Button(description='Update')
  back_button = widgets.Button(description='Back')

  def on_update_button_clicked(b):
    all_input_values = [new_pwd_textbox.value, retype_textbox.value]
    # check null values

    for i in all_input_values:
      if i == "":
        cpwd_msg_label.value = "❗Missing Value. Please input all items"
        return
      if not is_valid_pwd(i):
        cpwd_msg_label.value = "❗Password must be 6-30 characters only."
        return

    if new_pwd_textbox.value != retype_textbox.value:
      cpwd_msg_label.value = "❗Retype password is not the same as your new password! Please check again!"
      return

    # check whether new password is the same as the original one
    c.execute(
        """
        SELECT M_PW FROM MEMBER
        WHERE M_USERNAME = '{}'
        """.format(member_username_2.value)
    )
    original_pw = c.fetchone()[0]
    if original_pw == new_pwd_textbox.value:
      cpwd_msg_label.value = "❗New password cannot be the same as the current one!"
      return

    c.execute(
    """
    UPDATE MEMBER
    SET M_PW = '{}'
    WHERE M_USERNAME = '{}'
    """.format(new_pwd_textbox.value, member_username_2.value)
    )
    c.execute("commit")
    cpwd_msg_label.value = "✅ Successfully Updated!"
  update_button.on_click(on_update_button_clicked)

  def on_back_button_clicked(b):
    showPage("Login")
  back_button.on_click(on_back_button_clicked)

  line1HBox = widgets.HBox([cpwd_label])
  line2HBox = widgets.HBox([new_pwd_label,new_pwd_textbox])
  line3HBox = widgets.HBox([retype_label,retype_textbox])
  line4HBox = widgets.HBox([update_button, back_button])
  line5HBox = widgets.HBox([cpwd_msg_label])

  display(widgets.VBox([line1HBox,line2HBox,line3HBox,line4HBox,line5HBox]))

##############################################
############## Welcome Tab ################
##############################################

# Pending Tabs: Search Product, Shopping Cart, Purchase History
with tb_all.output_to('Welcome'):
  welcome_label = Label("Welcome! Start browsing with the following functions:")
  change_info_btn = widgets.Button(description='Manage My Account')
  search_prod_btn = widgets.Button(description='Search Products')
  shopping_cart_btn = widgets.Button(description='Shopping Cart')
  history_btn = widgets.Button(description='Purchase History')

  def on_change_info_btn_click(b):
    showPage('Update_Personal_Account')
  change_info_btn.on_click(on_change_info_btn_click)

  display(widgets.VBox([welcome_label,change_info_btn,search_prod_btn,shopping_cart_btn,history_btn]))

  def on_search_produc_btn_click(b):
    showPage('Search_Product')
  search_prod_btn.on_click(on_search_produc_btn_click)
  show_records()

##############################################
#### Update_Personal_Account Tab ##########
##############################################


with tb_all.output_to('Update_Personal_Account'):
  m_id_textbox = Text(description='Member_ID', disabled=True)
  up_username_label = Label("Username: ")
  up_username_textbox = Text(placeholder='Your Username', disabled=True)
  up_password_label = Label("Password: ")
  up_password_textbox = Text(placeholder='Your Password', disabled=True)
  up_firstname_label = Label("First Name: ")
  up_firstname_textbox = Text(placeholder='Tai Ming', disabled=True)
  up_lastname_label = Label("Last Name: ")
  up_lastname_textbox = Text(placeholder='Wong', disabled=True)
  up_address_label = Label("Address: ")
  up_address_textbox = Text(placeholder='Sai Kung, Hong Kong', disabled=True)
  up_gender_label = Label("Gender: ")
  up_gender_dropdown = widgets.Dropdown(options=["Male", "Female", "Others"],disabled=True)
  up_dob_label = Label("Date of Birth: ")
  up_dob_datepicker = DatePicker(placeholder='2000/01/01', disabled=True)
  up_phone_label = Label("Phone: ")
  up_phone_textbox = Text(placeholder='12345678', disabled=True)
  up_email_label = Label("Email: ")
  up_email_textbox = Text(placeholder='e.g, example@domain.com', disabled=True)
  up_creditno_label = Label("Credit No.: ")
  up_creditno_textbox = Text(placeholder='1234567891234567', disabled=True)
  up_security_question_label = Label("Security Question: ")
  up_cmb = widgets.Dropdown(options=["What is your favorite animal?", "Which city did you live when you were 10?",
                    "What year is your grandmother born?", "What is the name of your first pet?",
                    "What is your favorite sport?"],disabled=True)
  up_security_answer_label = Label("Security Answer: ")
  up_security_answer_textbox = Text(placeholder='Security Answer', disabled=True)
  up_expiry_date_label = Label("Expiry Date: ")
  up_expiry_date_datepicker = DatePicker(dplaceholder='1991-01-01', disabled=True)
  up_cvc_label = Label("CVC: ")
  up_cvc_textbox = Text(placeholder='000', disabled=True)

  up_status = Label("")
  retrieve_btn = widgets.Button(description = "Retrieve Info.")
  spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  update_btn = widgets.Button(description = "Update")
  up_back_btn = widgets.Button(description = "Back")

  # define retrieve info
  def on_retrieve_button_clicked(b):
    up_password_textbox.disabled = False
    up_firstname_textbox.disabled = False
    up_lastname_textbox.disabled = False
    up_address_textbox.disabled = False
    up_phone_textbox.disabled = False
    up_email_textbox.disabled = False
    up_creditno_textbox.disabled = False
    up_security_answer_textbox.disabled = False
    up_cvc_textbox.disabled = False
    up_gender_dropdown.disabled = False
    up_cmb.disabled = False
    up_expiry_date_datepicker.disabled = False
    up_gender_dropdown.disabled = False
    up_dob_datepicker.disabled = False

    if up_username_textbox == "":
      up_status.value = "❗Missing Username. Please input all items"
      return
    else:
      c.execute("SELECT * FROM MEMBER WHERE M_USERNAME= '{}' AND M_PW = '{}'".format(member_username.value, member_pwd.value))
      for row in c:
        m_id_textbox.value = str(row[0])
        up_username_textbox.value = row[3]
        up_password_textbox.value = row[4]
        up_firstname_textbox.value = row[1]
        up_lastname_textbox.value = row[2]
        up_address_textbox.value = row[5]
        up_gender_dropdown.value = row[14]
        up_dob_datepicker.value = row[13].date()
        up_phone_textbox.value = row[11]
        up_email_textbox.value = row[12]
        up_creditno_textbox.value = row[8]
        up_cmb.value = row[6]
        up_security_answer_textbox.value = row[7]
        up_expiry_date_datepicker.value =row[9].date()
        up_cvc_textbox.value = row[10]

  retrieve_btn.on_click(on_retrieve_button_clicked)

  # define update info
  def on_update_btn_clicked(b):

    all_input_values = [up_password_textbox.value, up_firstname_textbox.value, up_lastname_textbox.value, up_address_textbox.value,
             up_phone_textbox.value, up_email_textbox.value, up_creditno_textbox.value, up_cvc_textbox.value, up_security_answer_textbox.value]

    # check null values
    for i in all_input_values:
      if i == "":
        up_status.value = "❗Missing Value. Please input all items"
        return

    try:
      if not is_valid_pwd(up_password_textbox.value):
        up_status.value = "❗Password must be 6-30 characters only."
        return

      if not is_valid_name(up_firstname_textbox.value):
        up_status.value = "❗Firstname must be 1-50 characters only."
        return

      if not is_valid_name(up_lastname_textbox.value):
        up_status.value = "❗Lastname must be 1-50 characters only."
        return


      if not is_valid_address(up_address_textbox.value):
        up_status.value = "❗Address must be within 100 characters."
        return

      if not is_valid_phone(up_phone_textbox.value):
        up_status.value = "❗Phone must be 8 numbers only."
        return

      if not is_valid_email(up_email_textbox.value):
        up_status.value = "❗Invalid email format. Please use example@domain.com"
        return

      if not is_valid_creditno(up_creditno_textbox.value):
        up_status.value = "❗Credit card number must be 8-20 numbers only."
        return

      if not is_valid_cvc(up_cvc_textbox.value):
        up_status.value = "❗CVC must be 3-4 numbers only."
        return

      if not is_valid_answer(up_security_answer_textbox.value):
        up_status.value = "❗Security answer must be 1-50 characters and numbers only."
        return


      c.execute(
      """
      Update MEMBER
      SET M_USERNAME = '{}',
      M_PW = '{}',
      M_FIRST_NAME = '{}',
      M_LAST_NAME = '{}',
      M_ADDRESS = '{}',
      M_GENDER = '{}',
      M_DOB = TO_DATE('{}', 'YYYY-MM-DD'),
      M_PHONE = '{}',
      M_EMAIL = '{}',
      M_CREDIT_NO = '{}',
      M_QUESTION = '{}',
      M_ANSWER = '{}',
      M_CREDIT_EXPIRE = TO_DATE('{}', 'YYYY-MM-DD'),
      M_CREDIT_CVC = '{}'
      WHERE M_ID = {}
      """.format(up_username_textbox.value,up_password_textbox.value,up_firstname_textbox.value,up_lastname_textbox.value,up_address_textbox.value,up_gender_dropdown.value,
            up_dob_datepicker.value,up_phone_textbox.value,up_email_textbox.value,up_creditno_textbox.value,up_cmb.value,
            up_security_answer_textbox.value,up_expiry_date_datepicker.value,up_cvc_textbox.value,m_id_textbox.value)
      )
      c.execute("commit")

      up_status.value = "✅ Successfully Updated!"

      m_id_textbox.value = ""
      up_username_textbox.value = ""
      up_password_textbox.value = ""
      up_firstname_textbox.value = ""
      up_lastname_textbox.value = ""
      up_address_textbox.value = ""
      up_phone_textbox.value = ""
      up_email_textbox.value = ""
      up_creditno_textbox.value = ""
      up_security_answer_textbox.value=""
      up_cvc_textbox.value=""
      up_gender_dropdown.value = None
      up_cmb.value = None
      up_dob_datepicker.value = None
      up_expiry_date_datepicker.value = None

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      up_status.value = f"Database Error:{error_obj.message}"

  update_btn.on_click(on_update_btn_clicked)

  def on_back_btn_clicked(b):
    showPage("Welcome")
  up_back_btn.on_click(on_back_btn_clicked)


  line1HBox = widgets.HBox([m_id_textbox, retrieve_btn])
  line_label_VBox = widgets.VBox([up_username_label, up_password_label, up_firstname_label, up_lastname_label,
                  up_address_label,up_gender_label,up_dob_label,up_phone_label,up_email_label,
                  up_creditno_label,up_cvc_label,up_security_question_label,up_security_answer_label,up_expiry_date_label])
  line_textbox_VBox = widgets.VBox([up_username_textbox, up_password_textbox, up_firstname_textbox, up_lastname_textbox,
                  up_address_textbox,up_gender_dropdown,up_dob_datepicker,up_phone_textbox,up_email_textbox,
                  up_creditno_textbox,up_cvc_textbox,up_cmb,up_security_answer_textbox,up_expiry_date_datepicker])
  line2_HBox = widgets.HBox([line_label_VBox,line_textbox_VBox])
  line3_HBox = widgets.HBox([up_back_btn, update_btn])
  line4HBox = widgets.HBox([up_status])

  display(widgets.VBox([line1HBox,spacer,line2_HBox,line3_HBox,line4HBox]))


##############################################
############ Search_Product Tab ###########
##############################################
with tb_all.output_to("Search_Product"):

  # Allow member to search products by keyword
  search_product_title = Label("Search products by entering product id, name, or category")
  search_product_textbox = Text(placeholder="Enter product ID, name, or category", disabled = False)
  search_product_label = Label("Search by:")
  search_status_label = Label("")
  search_button = widgets.Button(description="Search",button_style='info', layout={'width': '100px', 'margin': '5px'})

  previous_5_button = widgets.Button(description="Previous 5 Items", button_style='info', layout={'width': '150px', 'margin': '5px'})
  next_5_button = widgets.Button(description="Next 5 Items", button_style='info', layout={'width': '150px', 'margin': '5px'})
  view_product_back_button = widgets.Button(description="Back")

  ret_5_id_textboxes = [Text(value='',description='Product ID:',placeholder='Not available',disabled=True) for i in range(5)]
  ret_5_name_textboxes = [Text(value='',description='Name:',placeholder='Not available',disabled=True) for i in range(5)]
  ret_5_category_textboxes = [Text(value='',description='Category:',placeholder='Not available',disabled=True) for i in range(5)]
  ret_5_price_textboxes = [Text(value='',description='Price:',placeholder='Not available',disabled=True) for i in range(5)]
  ret_5_detail_buttons = [widgets.Button(description='Product Details',disabled=True) for i in range(5)]
  ret_5_data_HBoxes = [widgets.HBox([ret_5_id_textboxes[i], ret_5_name_textboxes[i],ret_5_category_textboxes[i],
                    ret_5_price_textboxes[i], ret_5_detail_buttons[i]]) for i in range(5)]

  current_record_idx = widgets.Label("0")
  search_results = []
  all_records = []

  start_idx = 0
  total_records = 0

  def clear_product_textbox():
    for i in range(5):
      ret_5_id_textboxes[i].value = ''
      ret_5_name_textboxes[i].value = ''
      ret_5_price_textboxes[i].value = ''
      ret_5_category_textboxes[i].value = ''
      ret_5_detail_buttons[i].disabled = True

  def update_button_state():
    previous_5_button.disabled = start_idx == 0
    next_5_button.disabled = (start_idx + 5 >= total_records)

  def show_records():
    global total_records, all_records

    c.execute("SELECT P_ID, P_NAME, P_PRICE, P_CATEGORY FROM PRODUCT")
    all_records = c.fetchall()
    total_records = len(all_records)

    clear_product_textbox()

    for i in range(5):
        result_idx = start_idx + i
        if result_idx < total_records:
            row = all_records[result_idx]
            ret_5_id_textboxes[i].value = str(row[0])  # ID
            ret_5_name_textboxes[i].value = str(row[1])  # Name
            ret_5_category_textboxes[i].value = str(row[3])  # Category
            ret_5_price_textboxes[i].value = str(row[2])  # Price
            ret_5_detail_buttons[i].disabled = False

            ret_5_id_textboxes[i].layout.display = ''
            ret_5_name_textboxes[i].layout.display = ''
            ret_5_category_textboxes[i].layout.display = ''
            ret_5_price_textboxes[i].layout.display = ''
            ret_5_detail_buttons[i].layout.display = ''
        else:
            ret_5_id_textboxes[i].layout.display = 'none'
            ret_5_name_textboxes[i].layout.display = 'none'
            ret_5_category_textboxes[i].layout.display = 'none'
            ret_5_price_textboxes[i].layout.display = 'none'
            ret_5_detail_buttons[i].layout.display = 'none'

    update_button_state()
    search_status_label.value = f"Showing all records {start_idx + 1} - {min(start_idx + 5, total_records)} of {total_records}"

  def on_next_button_clicked(b):
    global start_idx
    if start_idx + 5 < total_records:
      start_idx += 5
      if search_product_textbox.value.strip():
        show_search_results()
      else:
        show_records()
  next_5_button.on_click(on_next_button_clicked)

  def on_prev_button_clicked(b):
    global start_idx
    if start_idx - 5 >= 0:
      start_idx -= 5
    else:
      start_idx = 0
    if search_product_textbox.value.strip():
      show_search_results()
    else:
      show_records()

  previous_5_button.on_click(on_prev_button_clicked)

  def on_view_back_button_clicked(b):
    showPage("Welcome")
    search_product_textbox.value = ''
  view_product_back_button.on_click(on_view_back_button_clicked)


  # Show product details
  def on_ret_5_detail_button_clicked(b):
    button_num = 0
    for button_num in range(len(ret_5_detail_buttons)):
      if ret_5_detail_buttons[button_num] == b:
        product_id_selected = ret_5_id_textboxes[button_num].value
        break

    dtl_id_textbox.value = str(product_id_selected)
    c.execute("SELECT * FROM PRODUCT WHERE P_ID = '{}'".format(product_id_selected))

    for row in c:
      dtl_name_textbox.value = row[1]
      dtl_id_textbox.value = str(row[0])
      dtl_price_textbox.value = str(row[2])
      dtl_description_textbox.value = row[3]
      dtl_color_textbox.value = row[4]
      dtl_capacity_textbox.value = str(row[5])
      dtl_category_textbox.value = str(row[9])

      if row[6] <= 0:
        dtl_stock_label.value = "❗ Not Available. Please contact service point."
      else:
        dtl_stock_label.value = "✅ Available"

    showPage("Show_Product_Detail")

  for dtl_btn in ret_5_detail_buttons:
    dtl_btn.on_click(on_ret_5_detail_button_clicked)


  def show_search_results():
    clear_product_textbox()
    global total_records
    total_records = len(search_results)

    if total_records == 0:
      search_status_label.value = "❗Item not found. Please re-enter another keyword."
      next_5_button.disabled = True
      previous_5_button.disabled = True


      for i in range(5):
        ret_5_id_textboxes[i].layout.display = 'none'
        ret_5_name_textboxes[i].layout.display = 'none'
        ret_5_category_textboxes[i].layout.display = 'none'
        ret_5_price_textboxes[i].layout.display = 'none'
        ret_5_detail_buttons[i].layout.display = 'none'

    else:
      search_status_label.value = f"Showing search results {start_idx + 1} - {min(start_idx + 5, total_records)} of {total_records}"  # Reset the message
      for i in range(5):
        result_idx = start_idx + i
        if result_idx < total_records:
          row = search_results[result_idx]
          ret_5_id_textboxes[i].value = str(row[0])
          ret_5_name_textboxes[i].value = str(row[1])
          ret_5_price_textboxes[i].value = str(row[3])
          ret_5_category_textboxes[i].value = str(row[2])
          ret_5_detail_buttons[i].disabled = False

          ret_5_id_textboxes[i].layout.display = ''
          ret_5_name_textboxes[i].layout.display = ''
          ret_5_category_textboxes[i].layout.display = ''
          ret_5_price_textboxes[i].layout.display = ''
          ret_5_detail_buttons[i].layout.display = ''
        else:
          ret_5_id_textboxes[i].layout.display = 'none'
          ret_5_name_textboxes[i].layout.display = 'none'
          ret_5_category_textboxes[i].layout.display = 'none'
          ret_5_price_textboxes[i].layout.display = 'none'
          ret_5_detail_buttons[i].layout.display = 'none'

      update_button_state()
      search_status_label.value = f"Showing search results {start_idx + 1} - {min(start_idx + 5, total_records)} of {total_records}"


  def on_search_clicked(b):
    global start_idx, search_results
    keyword = search_product_textbox.value.strip()

    if not keyword:
      start_idx = 0
      show_records()
      return

    if keyword.isnumeric():
      c.execute("SELECT P_ID, P_NAME, P_CATEGORY, P_PRICE FROM PRODUCT WHERE LOWER(P_NAME) LIKE :1 OR LOWER(P_CATEGORY) LIKE :2 OR P_ID = :3",
              ('%' + keyword + '%', '%' + keyword + '%', int(keyword),))
    else:
      c.execute("SELECT P_ID, P_NAME, P_CATEGORY, P_PRICE FROM PRODUCT WHERE LOWER(P_NAME) LIKE :1 OR LOWER(P_CATEGORY) LIKE :2",
              ('%' + keyword + '%','%' + keyword + '%',))

    search_results = c.fetchall()
    start_idx = 0
    show_search_results()


  search_button.on_click(on_search_clicked)

  ret_5_rcd_ctrl_HBox = widgets.HBox([previous_5_button, next_5_button])
  ret_5_VBox_items = [search_status_label] + ret_5_data_HBoxes + [ret_5_rcd_ctrl_HBox, view_product_back_button]

  ret_5_VBox = widgets.VBox(ret_5_VBox_items)
  display(widgets.VBox([search_product_title,
             widgets.HBox([search_product_label, search_product_textbox, search_button]),
             ret_5_VBox]))


show_records()


##############################################
##### Show Product Details Tab #########
##############################################


with tb_all.output_to("Show_Product_Detail"):
  dtl_id_label = Label("Product ID: ", layout ={'margin':'5px'})
  dtl_id_textbox = Text(value='', placeholder='Product ID', disabled=False, layout={'width': '200px', 'margin': '5px'})

  dtl_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  dtl_name_label = Label("Product Name: ", layout ={'margin':'5px'})
  dtl_name_textbox = Text(placeholder='Product Name:', disabled=False,layout={'width': '200px', 'margin': '5px'})

  dtl_price_label = Label("Price: ", layout ={'margin':'5px'})
  dtl_price_textbox = Text(placeholder='Price:', disabled=False,layout={'width': '200px', 'margin': '5px'})

  dtl_description_label = Label("Product Description: ", layout ={'margin':'5px'})
  dtl_description_textbox = Text(placeholder='Product Description:', disabled=False,layout={'width': '200px', 'margin': '5px'})

  dtl_color_label = Label("Color: ", layout ={'margin':'5px'})
  dtl_color_textbox = Text(placeholder='Color:', disabled=False,layout={'width': '200px', 'margin': '5px'})

  dtl_capacity_label = Label("Capacity: ", layout ={'margin':'5px'})
  dtl_capacity_textbox = Text(placeholder='Capacity:', disabled=False,layout={'width': '200px', 'margin': '5px'})

  dtl_category_label = Label("Category: ", layout ={'margin':'5px'})
  dtl_category_textbox = Text(placeholder='Category:', disabled=False,layout={'width': '200px', 'margin': '5px'})

  dtl_stock_label = Label(" ", layout={'width': '300px', 'margin': '5px'})

  dtl_item_count_label = Label("Purchase Number:", layout ={'margin':'5px'})
  dtl_item_count_intText = widgets.IntText(disabled=False,layout={'width': '200px', 'margin': '5px'})

  dtl_add_shopping_cart = widgets.Button(description = "Add to Shopping Cart", layout = {'width': '200px', 'margin': '5px'})
  dtl_back_button = widgets.Button(description = "Back")
  dtl_status_label = Label("")

  def on_dtl_back_button_clicked(b):
    showPage("Search_Product")
    show_records()
  dtl_back_button.on_click(on_dtl_back_button_clicked)

  # Put product into shopping cart
  # Fix insert command
  def on_dtl_add_shopping_cart_clicked(b):
    formatted_date = datetime.now().strftime('%Y-%m-%d')
    c.execute(
    # SC_ID, SC_ITEM_COUNT, SC_LAST_UPDATE, MEMBER_M_ID, SC_STATUS, PRODUCT_P_ID
    """INSERT INTO SHOPPINGCART VALUES ({},{}, TO_DATE('{}','YYYY-MM-DD'),'{}','{}',{})
    """.format("1", int(dtl_item_count_intText.value), formatted_date, int(mid_label.value), "pending", int(dtl_id_textbox.value))
    )
    c.execute("commit")
    dtl_add_shopping_cart.disabled = True
    dtl_status_label.value = "✅ Successfully added."
  dtl_add_shopping_cart.on_click(on_dtl_add_shopping_cart_clicked)


  # display boxes
  dtl_line1HBox = widgets.HBox([dtl_id_label, dtl_id_textbox, dtl_stock_label])
  dtl_line2HBox = widgets.HBox([dtl_spacer])
  dtl_labelVBox = widgets.VBox([dtl_name_label, dtl_price_label, dtl_description_label, dtl_color_label, dtl_capacity_label, dtl_category_label,dtl_item_count_label])
  dtl_textboxVBox = widgets.VBox([dtl_name_textbox, dtl_price_textbox, dtl_description_textbox, dtl_color_textbox, dtl_capacity_textbox, dtl_category_textbox, dtl_item_count_intText])
  dtl_line3HBox = widgets.HBox([dtl_labelVBox, dtl_textboxVBox])
  dtl_line4HBox = widgets.HBox([dtl_add_shopping_cart, dtl_back_button])

  display(widgets.VBox([dtl_line1HBox, dtl_line2HBox, dtl_line3HBox, dtl_line4HBox]))


showPage("Introduction")

In [ ]:
c.close()